In [24]:
# %pip install --upgrade --quiet langchain langchain-community chromadb bs4 boto3
# %pip install --upgrade --quiet pydantic
# %pip install --upgrade --quiet sentence-transformers

%pip install --upgrade --quiet sagemaker

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
# %pip install --upgrade --quiet trulens_eval
# %pip install --upgrade --quiet jinja2

In [4]:
%pip install --quiet ipywidgets>=8.0.6
# "ipython>=8.12.0"

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip show langchain

Name: langchain
Version: 0.1.14
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/ubuntu/.local/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: trulens-eval
Note: you may need to restart the kernel to use updated packages.


In [3]:
from typing import Dict
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json

sagemaker_endpoint_name = "mt-djl-ds-chatglm3-g4dn"

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        parameters = model_kwargs
        if 'temperature' not in parameters:
            parameters['temperature'] = 0.01
        input = {"inputs": prompt, "parameters": parameters }
        input_str = json.dumps(input).encode('utf-8')
        return input_str
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["outputs"]

chatglm_model = SagemakerEndpoint(
    endpoint_name=sagemaker_endpoint_name, 
    region_name="us-east-1", 
    content_handler=ContentHandler()
)


In [4]:
chatglm_model.invoke("Amazon Bedrock是什么?")

'Amazon Bedrock 是一种基于云的基础设施即服务(IaaS)工具，旨在为开发人员和运维团队提供一种可靠、可扩展的基础设施管理方式。它提供了一个统一的管理界面，让用户可以轻松管理 AWS 资源，包括计算、存储、网络、数据库等。Bedrock 的目标是简化 AWS 的基础设施管理，使用户能够更专注于应用程序的开发和部署，而不必担心底层基础设施的复杂性。\n\nBedrock 提供了一些核心功能，包括：\n\n1. 统一管理界面：Bedrock 提供了一个统一的控制台，让用户可以轻松管理 AWS 资源。\n2. 自动化：Bedrock 提供了自动化工具，可以帮助用户自动化各种基础设施管理任务，例如创建、更新和删除资源。\n3. 可扩展性：Bedrock 旨在提供一种可扩展的基础设施管理方式，可以适应各种规模和需求的场景。\n4. 安全：Bedrock 提供了各种安全工具和功能，帮助用户保护基础设施和数据安全。\n\n总的来说，Amazon Bedrock 是一种强大的基础设施管理工具，可以帮助用户更轻松地管理 AWS 资源，提高效率和可靠性。'

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='moka-ai/m3e-base')

In [6]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://huggingface.co/blog/zh/moe")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db_500chunk/")

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [ ]:
len(splits)

In [7]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser

vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_db_500chunk/")

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [8]:
questions = [
    "MOE有什么优势?",
    "为什么MOE的训练和推理的速度更快?",
    "简单总结MoE模型的特点?",
    "Shazeer 将 MoE 应用在了哪个应用领域?",
    "MoE模型的训练过程中，如何保证令牌不会总是被发送给少数几个受欢迎的专家?",
    "MoE模型训练时，怎么保证token的负载均衡?",
    "GShard如何保证MoE模型的负载均衡?",
    "MoE的专家数量达到多少以后，效率就会显著降低?",
    "怎么将MoE模型的推理阶段更加高效?",
    "在微调MoE这样的稀疏模型时，怎么防止出现过拟合?",
    "模型的并行有几种形式?",
    "部署 MoE 模型有哪些技术?"
]

In [11]:
results = vectorstore.similarity_search(questions[0], k=2)
for r in results:
    print(r.page_content)

[Switch Transformers paper](https://arxiv.org/abs/2101.03961) 论文中的 MoE layer

总结来说，在混合专家模型 (MoE) 中，我们将传统 Transformer 模型中的每个前馈网络 (FFN) 层替换为 MoE 层，其中 MoE 层由两个核心部分组成: 一个门控网络和若干数量的专家。
尽管混合专家模型 (MoE) 提供了若干显著优势，例如更高效的预训练和与稠密模型相比更快的推理速度，但它们也伴随着一些挑战:
了解了 MoE 的基本概念后，让我们进一步探索推动这类模型发展的研究。





		混合专家模型简史
	

混合专家模型 (MoE) 的理念起源于 1991 年的论文 Adaptive Mixture of Local Experts。这个概念与集成学习方法相似，旨在为由多个单独网络组成的系统建立一个监管机制。在这种系统中，每个网络 (被称为“专家”) 处理训练样本的不同子集，专注于输入空间的特定区域。那么，如何选择哪个专家来处理特定的输入呢？这就是门控网络发挥作用的地方，它决定了分配给每个专家的权重。在训练过程中，这些专家和门控网络都同时接受训练，以优化它们的性能和决策能力。
在 2010 至 2015 年间，两个独立的研究领域为混合专家模型 (MoE) 的后续发展做出了显著贡献:


In [12]:
from langchain.prompts import PromptTemplate

prompt_template = """请基于以下的3引号里面的内容，简洁、准确的回答最后的问题，并使用中文。如果你无法从已知内容获取答案，就直接返回'根据已知信息无法回答该问题。'，不要编造答案。

已知内容:
```
{context}
```

问题: {question}
答案:"""


In [13]:
prompt_template.format(question='qqqqq', context= 'xcccccc')

"请基于以下的3引号里面的内容，简洁、准确的回答最后的问题，并使用中文。如果你无法从已知内容获取答案，就直接返回'根据已知信息无法回答该问题。'，不要编造答案。\n\n已知内容:\n```\nxcccccc\n```\n\n问题: qqqqq\n答案:"

In [14]:
from trulens_eval import Tru

# initialize evaluation db
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [15]:
from trulens_eval.tru_custom_app import instrument

class RAG_App:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vectorstore.similarity_search(query, k=5)
        docs = [doc.page_content for doc in results]
        return docs

    @instrument
    def generate_completion(self, query: str, context: list) -> str:
        """
        Generate answer from context.
        """
        context_str = "\n\n".join(context)
        p = prompt_template.format(question=query, context=context_str)
        return chatglm_model(p)

    @instrument
    def query(self, query: str) -> str:
        context_slist = self.retrieve(query)
        completion = self.generate_completion(query, context_slist)
        return completion

rag = RAG_App()

In [54]:
from trulens_eval import Bedrock
from typing import Dict, Optional, Sequence

class Claude3_Bedrock(Bedrock):
    def _create_chat_completion(
        self,
        prompt: Optional[str] = None,
        messages: Optional[Sequence[Dict]] = None,
        **kwargs
    ) -> str:
        assert self.endpoint is not None

        import json

        print("prompt", prompt)
        print("messages", messages)

        if messages:
            pass
        elif prompt:
            messages = [{
                "role": "user",
                "content": prompt
            }]
        else:
            raise ValueError("Either 'messages' or 'prompt' must be supplied.")
        
        body = json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 2048,
                "temperature": 0.01,
                "messages": messages
            }
        )

        accept = "application/json"
        content_type = "application/json"

        response = self.endpoint.client.invoke_model(
            body=body, modelId=self.model_id
        )

        response_body = json.loads(response.get('body').read()).get('content')[0]['text']

        return response_body

In [55]:
fclaude = Claude3_Bedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", region_name='us-east-1')

In [56]:
from trulens_eval import Feedback, Select, Bedrock

from trulens_eval.feedback import Groundedness
import numpy as np

grounded = Groundedness(groundedness_provider=fclaude)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(fclaude.relevance_with_cot_reasons, name = "Answer Relevance").on_input_output()

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fclaude.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag, app_id='ChatGLM_500Chunk_Sonnet', feedbacks=[f_groundedness, f_qa_relevance, f_context_relevance])

In [60]:
with tru_rag as recording:
    rag.query(questions[5])

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
# with tru_rag as recording:
#     for question in questions:
#         rag.query(question)

In [21]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
ChatGLM_500Chunk,1.0,1.0,1.0,2.0,0.0


In [ ]:
# with tru_recorder as recording:
#     for question in questions:
#         rag_chain.invoke(question)

In [22]:
tru.run_dashboard()
# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://172.31.20.134:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
tru.reset_database()

In [27]:
from langchain_community.chat_models import BedrockChat
import boto3

session = boto3.Session()
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'
)
model_kwargs = {
    "max_tokens": 2048,
    "temperature": 0.01,
    # "top_k": 250,
    # "top_p": 1,
}

claude3_llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    model_kwargs=model_kwargs
)

In [28]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    HumanMessage(content="星际穿越这部电影怎么样?")
]
claude3_llm(messages)

/home/ubuntu/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='星际穿越是一部非常出色的科幻电影,从多个方面都值得一看:\n\n1. 科学性强 - 电影中涉及到的相对论、黑洞、高维度等概念都有一定的科学依据,导演与著名物理学家科普作家合作,努力让科学内容尽可能贴近现有理论。\n\n2. 视觉效果震撼 - 太空旅行、异次元空间等场景的视觉呈现非常梦幻而宏大,给人深刻的视觉冲击。\n\n3. 剧情设置巧妙 - 人类面临资源枯竭的未来,科学家们孤注一掷寻找新家园的故事情节紧凑且富有张力。\n\n4. 探讨哲理命题 - 影片蕴含了对爱、家庭、牺牲、人性等命题的思考,给人以启迪。\n\n5. 演员阵容出色 - 马修·麦康纳、安妮·海瑟薇等主演都有精彩的演绎。\n\n总的来说,这部电影在视觉体验和思想内涵上都达到了很高的水准,是一部值得反复观赏的佳作。你看过这部影片吗?有何其他感想?', response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 22, 'completion_tokens': 396, 'total_tokens': 418}}, id='run-8dbbbbe1-955d-4779-bcdc-4ecc687d0311-0')

In [30]:
# 与上面的调用一样，后面会使用message API调用
claude3_llm.invoke("星际穿越这部电影怎么样?")

AIMessage(content='星际穿越是一部非常出色的科幻电影,从多个方面都值得一看:\n\n1. 科学性强 - 电影中涉及到的相对论、黑洞、高维度等概念都有一定的科学依据,导演与著名物理学家科普作家合作,努力让科学内容尽可能贴近现有理论。\n\n2. 视觉效果震撼 - 太空旅行、异次元空间等场景的视觉呈现非常梦幻而宏大,给人深刻的视觉冲击。\n\n3. 剧情设置巧妙 - 人类为了生存不得不去寻找新家园的故事情节引人入胜,加上亲情的元素也很打动人心。\n\n4. 哲理性强 - 影片蕴含了对时间、爱、生命等哲学命题的思考,给人以启迪。\n\n5. 演员阵容强大 - 马修·麦康纳、安妮·海瑟薇等主演都演技出众。\n\n总的来说,这部电影在娱乐性与思考性之间取得了很好的平衡,是一部值得细细品味的佳作。你看过这部影片吗?有何其他感想?', response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 22, 'completion_tokens': 381, 'total_tokens': 403}}, id='run-25cd6003-5bf3-4644-aa31-98bb0399394f-0')

In [13]:
from langchain.globals import set_verbose
set_verbose(True)

In [32]:
rag.query("MOE有什么优势?")

'根据已知信息，混合专家模型（MoE）的优势包括：1. 更高效的预训练，2. 与稠密模型相比，推理速度更快。'

In [71]:
rag_chain.invoke("为什么MOE的训练和推理的速度更快?", )

<class 'langchain_core.documents.base.Document'>


'根据 Switch Transformers 论文中的描述，混合专家模型 (MoE) 通过将传统 Transformer 模型中的每个前馈网络 (FFN) 层替换为 MoE 层，其中 MoE 层由两个核心部分组成: 一个门控网络和若干数量的专家。这种替换可以提高训练和推理速度。具体来说，由于 MoE 模型只使用其中的一部分参数，所以在推理过程中只需要加载部分参数到内存中，从而降低了内存需求。此外，由于 MoE 模型中的专家数量相对较少，因此训练和推理过程中计算量更小，从而提高了速度。'

In [77]:
rag_chain.invoke("简单总结MoE模型的特点")

<class 'langchain_core.documents.base.Document'>


'MoE模型是一种混合专家模型的简写，它将传统Transformer模型中的每个前馈网络（FFN）层替换为MoE层，其中MoE层由两个核心部分组成：一个门控网络和若干数量的专家。MoE模型具有以下特点：1. 训练效率高，能够实现更高效的计算预训练；2. 与稠密模型相比，推理速度更快；3. 具有较好的泛化能力，但在微调阶段面临泛化能力不足的问题；4. 参数需求较大，推理过程中只使用其中一部分，需要将所有参数加载到内存中，因此对内存的需求较高。'

In [87]:
rag_chain.invoke("Shazeer将 MoE 应用在了哪个应用领域？")

<class 'langchain_core.documents.base.Document'>


'Shazeer 将 MoE 应用于自然语言处理（NLP）领域。'

In [91]:
rag_chain.invoke("MoE模型的训练过程中，如何保证令牌不会总是被发送给少数几个受欢迎的专家？")

'ST-MoE 论文中显示了哪些令牌组被发送给了哪个专家的表格。根据文中所述，混合专家模型 (MoE) 中的令牌会根据专家的权重比例被分配，以保证所有专家接收到大致相等数量的训练样本，从而平衡专家之间的选择。此外，辅助损失可以被用来鼓励给予所有专家相同的重要性，以避免令牌总是被发送给少数几个受欢迎的专家。'

In [92]:
rag_chain.invoke("MoE模型训练时，怎么保证token的负载均衡？")

'根据所提供的信息，MoE模型在训练时通过令牌路由和负载均衡机制来保证token的负载均衡。具体来说，该模型将令牌发送到拥有所需专家的节点，每个节点处理不同批次的训练样本。这种设计可以确保每个专家接收到不同数量的令牌，从而实现token的负载均衡。此外，该模型还采用了容量因子来提高模型性能，但这也意味着更高的通信成本和对保存激活值的显存的需求。在设备通信带宽有限的情况下，选择较小的容量因子可能是更佳的策略。'

In [93]:
rag_chain.invoke("GShard如何保证MoE模型的负载均衡？")

'GShard 保证 MoE 模型的负载均衡主要通过以下几个方面：\n1. 随机路由：在 Top-2 设置中，我们始终选择排名最高的专家，但第二个专家是根据其权重比例随机选择的。\n2. 专家容量：可以设定一个阈值，定义一个专家能处理多少令牌。如果两个专家的容量都达到上限，令牌就会溢出，并通过残差连接传递到下一层，或在某些情况下被完全丢弃。\n3. 辅助损失：为了鼓励给予所有专家相同的重要性，引入了一个辅助损失，确保所有专家接收到大致相等数量的训练样本，从而平衡了专家之间的选择。\n通过这些措施，GShard 能够实现 MoE 模型的负载均衡，从而提高训练效率。'

In [97]:
rag_chain.invoke("MoE的专家数量达到多少以后，效率就会显著降低？")

'根据所提供的信息，混合专家模型（MoE）的专家数量达到一定程度后，效率会显著降低。具体来说，当专家数量超过一定程度（约为1000-2000）时，训练和推理速度会明显变慢。因此，在选择专家数量时，需要权衡模型性能和计算资源的使用。'

In [99]:
chatglm_model("MoE的专家数量达到多少以后，效率就会显著降低？")

'这是一个比较复杂的问题，因为机器学习模型的效率受到很多因素的影响，包括数据集大小、模型结构、超参数选择等等。同时，专家数量也不是一个固定的值，而是需要根据具体问题进行调整。\n\n一般来说，当专家数量达到一定程度时，模型的效率可能会出现下降趋势。这是因为随着专家数量的增加，模型的复杂度也会增加，导致模型训练时间增加，同时过拟合的风险也会增加。但是，具体多少数量的专家会使得效率显著降低，需要根据具体问题和数据集情况进行实验和分析。\n\n另外，还有一些技巧可以帮助提高模型效率，例如正则化、早停、Dropout等。这些技巧可以在一定程度上减少过拟合现象，提高模型的泛化能力，同时也可以降低模型的复杂度，提高训练效率。'

In [100]:
rag_chain.invoke("在微调MoE这样的稀疏模型时，怎么防止出现过拟合？")

'根据ST-MoE论文，微调稀疏混合专家模型（MoE）时，可以通过以下方法防止过拟合：\n\n1. 降低学习率和调大批量可以提升稀疏模型微调质量。\n2. 尝试冻结所有非专家层的权重，实践中，这会导致性能大幅下降，但这符合预期，因为混合专家模型（MoE）层占据了网络的主要部分。可以尝试相反的方法：仅冻结MoE层的参数。实验结果显示，这种方法几乎与更新所有参数的效果相当。这种做法可以加速微调过程，并降低显存需求。\n3. 考虑特别的微调超参数设置，例如，稀疏模型往往更适合使用较小的批量大小和较高的学习率，这样可以获得更好的训练效果。'

In [102]:
rag_chain.invoke("怎么将MoE模型的推理阶段更加高效？")

'将MoE模型的推理阶段更加高效的方法包括：\n\n1. 优化门控网络和专家的权重训练过程，以提高模型在微调阶段的泛化能力。\n2. 采用更高效的计算方法，例如将稀疏混合专家模型 (SMoE) 蒸馏回具有更少实际参数但相似等价参数量的稠密模型。\n3. 探索合并专家模型的技术及其对推理时间的影响。\n4. 尝试对Mix'

In [103]:
rag_chain.invoke("模型的并行有几种形式？")

'问题: 模型的并行有几种形式？\n答案: 数据并行、模型并行、模型和数据并行、专家并行。'

In [104]:
rag_chain.invoke("部署 MoE 模型有哪些技术?")

'部署 MoE 模型有以下几种技术：\n\n1. 预先蒸馏 (Pre-distillation)：将 MoE 模型蒸馏回其对应的稠密模型，以保留稀疏性带来的性能提升，同时使得在推理中使用更小型的模型成为可能。\n\n2. 任务级别路由 (Task-level routing)：将整个句子或任务直接路由到一个专家，以提取出一个用于服务的子网络，有助于简化模型的结构。\n\n3. 专家网络聚合 (Expert network aggregation)：通过合并各个专家的权重，在推理时减少了所需的参数数量，在不显著牺牲性能的情况下降低模型的复杂度。'